<a href="https://colab.research.google.com/github/IrynaBarnych/AI_23/blob/hw/module7/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D1%87%D0%BD%D0%B0_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Завдання

* Створіть Reader
* Створіть датасет та розділіть його на тренувальні та тестові дані
* Виберіть метрики для поріняння якості моделей
* На основі метрик виберіть найкращу модель
* Використайте `optuna` для підбору найкращих параметрів



In [16]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module7/data/Blog%20Ratings.csv")

df.head()

,blog_id,userId,ratings
0,9025,11,3.5
1,9320,11,5.0
2,9246,11,3.5
3,9431,11,5.0
4,875,11,2.0


In [17]:
df.describe()

,blog_id,userId,ratings
count,200140.000000,200140.000000,200140.000000
mean,5652.533621,2545.710158,3.117468
std,2970.685946,1446.195478,1.768113
min,1.000000,10.000000,0.500000
25%,2906.000000,1314.000000,2.000000
50%,5994.000000,2552.000000,3.500000
75%,8510.000000,3795.000000,5.000000
max,9755.000000,5010.000000,5.000000


In [18]:
!pip install -q surprise

In [19]:
!pip install -q optuna

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection, metrics
import lightgbm as lgb
import optuna


In [21]:
from surprise import Dataset, SVD, Reader

reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df[["userId","blog_id","ratings"]], reader)

In [22]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, train_size=0.8)

In [23]:
from surprise import BaselineOnly

bsl_options = {'method': 'als',
               'reg_u': 0.0001,
               'reg_i': 0.0001}

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

Estimating biases using als...


In [24]:
from surprise import SVD

alfo = SVD(n_factors=100,
           n_epochs=50,
           )

algo.fit(trainset)

Estimating biases using als...


In [25]:
from surprise import KNNBasic

algo = KNNBasic(k=10,
                min_k=1,
                sim_options={'name': 'cosine',
                             'user_based': True})

algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [33]:
from surprise import CoClustering

algo = CoClustering(n_cltr_u = 5,
                    n_cltr_i = 15,
                    n_epochs = 40)

algo.fit(trainset)


In [32]:
from surprise import BaselineOnly, accuracy


bsl_options = {'method': 'als',
               'reg_u': 0.0001,
               'reg_i': 0.0001}

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(trainset)

preds = algo.test(testset)

print(f"mae = {accuracy.mae(preds, verbose=False)}")
print(f"mse = {accuracy.mse(preds, verbose=False)}")
print(f"rmse= {accuracy.rmse(preds, verbose=False)}")
print(f"fcp = {accuracy.fcp(preds, verbose=False)}")

Estimating biases using als...
mae = 1.6460378760573657
mse = 3.455195713515599
rmse= 1.8588156749703826
fcp = 0.49902208881895616


In [34]:
algo.predict(uid=2,
             iid=97)

Prediction(uid=2, iid=97, r_ui=None, est=3.117111771759768, details={'was_impossible': False})

In [35]:
import optuna
from sklearn import metrics

def objective(trial):

    bsl_options = {
        'method': trial.suggest_categorical('method', ['als', 'sgd']),
        'reg_u': trial.suggest_loguniform('reg_u', 1e-10, 1e-2),
        'reg_i': trial.suggest_loguniform('reg_i', 1e-10, 1e-2)
    }


    model = BaselineOnly(bsl_options=bsl_options)


    model.fit(trainset)

    preds = model.test(testset)
    mae = accuracy.mae(preds, verbose=False)

    return mae

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)


study.best_params

[I 2024-05-21 19:50:22,008] A new study created in memory with name: no-name-76403acd-2a6c-48df-8fdc-e5ec50a882d5
<ipython-input-35-f9057487933f>:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_u': trial.suggest_loguniform('reg_u', 1e-10, 1e-2),
<ipython-input-35-f9057487933f>:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'reg_i': trial.suggest_loguniform('reg_i', 1e-10, 1e-2)


Estimating biases using sgd...


[I 2024-05-21 19:50:23,734] Trial 0 finished with value: 1.629309147603119 and parameters: {'method': 'sgd', 'reg_u': 2.339152739340192e-07, 'reg_i': 0.0006359226451402258}. Best is trial 0 with value: 1.629309147603119.


Estimating biases using als...


[I 2024-05-21 19:50:24,811] Trial 1 finished with value: 1.6460375915143788 and parameters: {'method': 'als', 'reg_u': 3.953705247926772e-06, 'reg_i': 0.00020067583377731082}. Best is trial 0 with value: 1.629309147603119.


...

Estimating biases using als...


[I 2024-05-21 19:50:35,162] Trial 8 finished with value: 1.6460386296619078 and parameters: {'method': 'als', 'reg_u': 2.745218014836116e-07, 'reg_i': 3.286366299149791e-06}. Best is trial 0 with value: 1.629309147603119.


Estimating biases using als...


[I 2024-05-21 19:50:36,424] Trial 9 finished with value: 1.646029994757483 and parameters: {'method': 'als', 'reg_u': 0.0032670309123320825, 'reg_i': 4.315381542214338e-05}. Best is trial 0 with value: 1.629309147603119.


{'method': 'sgd',
 'reg_u': 2.339152739340192e-07,
 'reg_i': 0.0006359226451402258}

In [36]:
print(study.best_params)

{'method': 'sgd', 'reg_u': 2.339152739340192e-07, 'reg_i': 0.0006359226451402258}


In [37]:
best_params = study.best_params

model = lgb.LGBMRegressor(**best_params,
                          n_estimator = 100,
                          verbose=-1)

model.fit(X_train, y_train,
          eval_set=[(X_test, y_test), (X_train, y_train)],
          eval_metric='mae')


NameError: name 'X_train' is not defined